In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from scipy.special import rel_entr

# Fixed parameters

In [2]:
samplesize = 5000
nbins = 75
width = 1.0/nbins

# Variable parameters (qubits and layers)

In [3]:
#List of number of qubits
qubits = [3]

#List of number of layers
layers = [5]

# Multiple layer fidelity histogram

In [4]:
def fidel_histogram(binsnumb, samples, circuit, nlayers):
    fidel_vector = []
    for i in range(samples):
        #angles1 = np.array([np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
        #             requires_grad=True)
        #angles2 = np.array([np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
        #             requires_grad=True)
        
        angles1 = np.array([ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)], 
                            [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] 
                            for j in range(nlayers)], requires_grad=True)
        
        angles2 = np.array([ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)], 
                            [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] 
                            for j in range(nlayers)], requires_grad=True)
        
        state1 = circuit(angles1)
        state2 = circuit(angles2)
        
        F = np.abs( np.dot(state1, state2) )**2
        
        fidel_vector.append(F)
        
    fidel_vector = np.asarray(fidel_vector)
    
    binsize = 1.0/binsnumb
    bins = np.arange(0, 1. + binsize, binsize)
    
    histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]
    
    return histogram, bins

# Code that executes the different conditions

In [5]:
now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")


for wires in qubits:
    dev = qml.device("default.qubit", wires=wires, shots=None)
        
    
    for nlayers in layers:
        
        #No connections
        def layer_noconnec(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
        
        def noconnec(rotations):
            qml.layer(layer_noconnec, nlayers, rotations)
            return qml.state()

        noconnec_circuit = qml.QNode(noconnec, dev)
        
        
        
        #Linear
        def layer_linear(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")

        def linear(rotations):
            qml.layer(layer_linear, nlayers, rotations)
            return qml.state()

        linear_circuit = qml.QNode(linear, dev)
        
        
        
        #Ring
        def layer_ring(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
            qml.CNOT(wires=[wires-1,0])
    
        def ring(rotations):
            qml.layer(layer_ring, nlayers, rotations)
            return qml.state()

        ring_circuit = qml.QNode(ring, dev)
        
        
        
        noconnec_hist, bins = fidel_histogram(nbins, samplesize, noconnec_circuit, nlayers)
        linear_hist, bins = fidel_histogram(nbins, samplesize, linear_circuit, nlayers)
        ring_hist, bins = fidel_histogram(nbins, samplesize, ring_circuit, nlayers)
        
            
        #Haar histogram for 'wires' qubits
        N = 2**wires #dimension of the Hilbert space
        histogram_Haar = []
        for i in range(nbins):
            histogram_Haar.append(  (1-bins[i])**(N-1) - (1-bins[i+1])**(N-1)  )
        
        
        D_KL_noconnec = sum( rel_entr(noconnec_hist, histogram_Haar) )
        D_KL_linear = sum( rel_entr(linear_hist, histogram_Haar) )
        D_KL_ring = sum( rel_entr(ring_hist, histogram_Haar) )
        
        
        f = open("expressibility_ansatz1_qubits{0}_layers{1}_samples{2}.txt".format(wires,nlayers, samplesize), "w")

        print('Circuit,Expressibility', file = f)
        print('No connections,', D_KL_noconnec, file = f)
        print('Linear,', D_KL_linear, file = f)
        print('Ring,', D_KL_ring, file = f)
        
        print('Execution: qubits=',wires,'; layers=', nlayers)
        print()
        
        
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Execution: qubits= 3 ; layers= 5

Tempo inicial: 
2023-07-10 16:04:20
Tempo final:
2023-07-10 16:16:16


In [6]:
f.close()